In [24]:
import pandas as pd
import numpy as np
import re
import string 
!pip install more-itertools
import itertools
import collections

### Load : Train and Test Data

In [25]:
train_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/HW1/train/train1.csv")
train_data.info()
train_data.describe()
text_train, y_train = train_data['text'], train_data['Target']
target_catagory_counts = y_train.value_counts()
print(target_catagory_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7320 entries, 0 to 7319
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      7320 non-null   int64 
 1   text    7320 non-null   object
 2   Target  7320 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 171.7+ KB
-1    4566
 0    1536
 1    1218
Name: Target, dtype: int64


In [3]:
text_train

0       @USAirways  ! THE WORST in customer service. @...
1       @united call wait times are over 20 minutes an...
2       @JetBlue what's up with the random delay on fl...
3       @AmericanAir Good morning!  Wondering why my p...
4       @united UA 746. Pacific Rim and Date Night cut...
                              ...                        
7315                              @AmericanAir followback
7316    @united thanks for the help. Wish the phone re...
7317    @usairways the. Worst. Ever. #dca #customerser...
7318    @nrhodes85: look! Another apology. DO NOT FLY ...
7319    @united you are by far the worst airline. 4 pl...
Name: text, Length: 7320, dtype: object

In [26]:
test_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/HW1/train/test1.csv")
test_data.info()
test_data.describe()
text_test = test_data['text']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7320 entries, 0 to 7319
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7320 non-null   int64 
 1   text    7320 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.5+ KB


In [16]:
text_test

0       @AmericanAir In car gng to DFW. Pulled over 1h...
1       @AmericanAir after all, the plane didnÂ‰Ã›Âªt ...
2       @SouthwestAir can't believe how many paying cu...
3       @USAirways I can legitimately say that I would...
4       @AmericanAir still no response from AA. great ...
                              ...                        
7315    @JetBlue Traveling with two kids tomorrow (age...
7316    @JetBlue Tx for the info. Just don't understan...
7317    @AmericanAir I understand. But why is this the...
7318                                 @USAirways really!??
7319    @united no I did not make connection.  Your st...
Name: text, Length: 7320, dtype: object

### Preprocessing

#### Tokenizer, Stemmer, Lemmatizer and Remove stop words

In [27]:
## Remove Non ascii characters from tweet
text_train = text_train.str.encode('ascii', 'ignore').str.decode('ascii')

## Remove Tweeter handlers from the tweet and HTTP URLs
text_train = text_train.str.replace('@[A-Za-z0-9]+','')
text_train = text_train.str.replace('https?://[A-Za-z0-9./]+','')

##Remove only numeric tokens
text_train = text_train.str.replace('[0-9]+','')

##sentence Tokenizer
#from nltk import sent_tokenize
#text_train = text_train.apply(lambda row: sent_tokenize(row.lower()))

## Remove pactuations from the tweet words
text_train = text_train.str.replace('[{}]'.format(string.punctuation),'')

## Lower case & Word Tokenizer
import nltk
nltk.download('punkt')
from nltk import word_tokenize
text_train = text_train.apply(lambda row: word_tokenize(row.lower()))

##Stemming
from nltk.stem.porter import PorterStemmer 
def stemming(data):
  out_sentence = [PorterStemmer().stem(token) for token in data]
  return out_sentence
text_train = text_train.apply(lambda row: stemming(row))

##lemmatizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lemmatizer(data):
  lemmatizer = WordNetLemmatizer()  
  out_sentence = [lemmatizer.lemmatize(token) for token in data]
  return(out_sentence)
text_train = text_train.apply(lambda row: lemmatizer(row))

## Remove Stop Words
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
def stopword(data):
  stop = set(nltk_stopwords.words('english'))
  wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
  for word in wh_words: 
      stop.remove(word)
  out_sentence = " ".join([token for token in data if token not in stop])
  return(out_sentence)
text_train = text_train.apply(lambda row: stopword(row))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
text_train

0       worst custom servic call month book flight poo...
1           call wait time minut airport wait time longer
2           what random delay flight ani chanc fals alarm
3       good morn wonder whi pretsa check wa board pas...
4       ua pacif rim date night cut constantli randoml...
                              ...                        
7315                                           followback
7316              thank help wish phone rep could accomid
7317                        worst ever dca customerservic
7318                                look anoth apolog fli
7319    far worst airlin plane delay round trip flight...
Name: text, Length: 7320, dtype: object

In [28]:
## Remove Non ascii characters from tweet
text_test = text_test.str.encode('ascii', 'ignore').str.decode('ascii')

## Remove Tweeter handlers from the tweet and HTTP URLs
text_test = text_test.str.replace('@[A-Za-z0-9]+','')
text_test = text_test.str.replace('https?://[A-Za-z0-9./]+','')

##Remove only numeric tokens
text_test = text_test.str.replace('[0-9]+','')

##sentence Tokenizer
#from nltk import sent_tokenize
#text_test = text_test.apply(lambda row: sent_tokenize(row.lower()))

## Remove pactuations from the tweet words
text_test = text_test.str.replace('[{}]'.format(string.punctuation),'')

## Lower case & Word Tokenizer
from nltk import word_tokenize
text_test = text_test.apply(lambda row: word_tokenize(row.lower()))

##Stemming
from nltk.stem.porter import PorterStemmer 
def stemming(data):
  out_sentence = [PorterStemmer().stem(token) for token in data]
  return out_sentence
text_test = text_test.apply(lambda row: stemming(row))

##lemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lemmatizer(data):
  lemmatizer = WordNetLemmatizer()  
  out_sentence = [lemmatizer.lemmatize(token) for token in data]
  return(out_sentence)
text_test = text_test.apply(lambda row: lemmatizer(row))

## Remove Stop Words
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
def stopword(data):
  stop = set(nltk_stopwords.words('english'))
  wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
  for word in wh_words: 
      stop.remove(word)
  out_sentence = " ".join([token for token in data if token not in stop])
  return(out_sentence)
text_test = text_test.apply(lambda row: stopword(row))

## Remove in frequent words and too many frequent words 


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
text_test

0       car gng dfw pull hr ago veri ici road onhold a...
1       plane didnt land ident wors condit grk accord ...
2       cant believ how mani pay custom left high dri ...
3       legitim say would rather driven cross countri ...
4                          still respons aa great job guy
                              ...                        
7315    travel two kid tomorrow age domest need birth ...
7316    tx info dont understand whi couldnt accur esti...
7317    understand whi thi onli flight day go twice im...
7318                                               realli
7319    make connect stellar employe vicki thoma refus...
Name: text, Length: 7320, dtype: object

#### Feature Selection : CountVectorizer, TfidfVectorizer, Bag of Word, HashingVectorizer

In [29]:
#### CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
Cvect = CountVectorizer(analyzer='word', ngram_range=(1,2))
X_train_cvect= Cvect.fit_transform(text_train)
X_test_cvect = Cvect.transform(text_test)
#print(Cvect.get_feature_names())
print(len(Cvect.get_feature_names()))

49659


In [ ]:
## Feature selection - Bag of Word

from sklearn.feature_extraction.text import CountVectorizer
bowvect = CountVectorizer(analyzer='word')
X_train_bow = bowvect.fit_transform(text_train).toarray()
X_test_bow = bowvect.transform(text_test).toarray()

In [31]:
#### TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
Tvect = TfidfVectorizer(analyzer='word')
X_train_tvect = Tvect.fit_transform(text_train)
X_test_tvect = Tvect.transform(text_test)
#print(Tvect.get_feature_names())
print(len(Tvect.get_feature_names()))


6659


In [10]:
### HashingVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
hvect = HashingVectorizer(analyzer='word', ngram_range=(1, 2), n_features=30000)
X_train_hvect = hvect.fit_transform(text_train)
X_test_hvect = hvect.transform(text_test)
print(X_train_hvect.shape)

(7320, 30000)


### ML Model

#### Logistic Regression

In [11]:
########## LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'C': [0.8,0.9,1.0,1.1,1.2]}
grid = GridSearchCV(LogisticRegression(max_iter=200,penalty= 'l2'), param_grid, cv=5, scoring='f1_macro')

In [17]:
### Save results in list
amsl={'PreProcessing':[],'ML_Model':[],'Mean_Score':[]}
allmodelscorelist = pd.DataFrame(amsl)

In [34]:
## CountVectorizer
lg_cv = grid.fit(X_train_cvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'CountVectorizer','ML_Model':'LogisticRegression','Mean_Score':grid.best_score_}, ignore_index=True) 

Best cross-validation f1_macro score: 0.7163
Best parameters:  {'C': 1.2}


In [35]:
## TfidfVectorizer
lg_tf = grid.fit(X_train_tvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'TfidfVectorizer','ML_Model':'LogisticRegression','Mean_Score':grid.best_score_}, ignore_index=True) 


Best cross-validation f1_macro score: 0.6810
Best parameters:  {'C': 1.2}


In [37]:
## Bag of Word
lg_bow = grid.fit(X_train_bow, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'Bag of Word','ML_Model':'LogisticRegression','Mean_Score':grid.best_score_}, ignore_index=True) 


Best cross-validation f1_macro score: 0.7163
Best parameters:  {'C': 1.2}


In [19]:
## HashingVectorizer
lg_hv = grid.fit(X_train_hvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'HashingVectorizer','ML_Model':'LogisticRegression','Mean_Score':grid.best_score_}, ignore_index=True) 


Best cross-validation f1_macro score: 0.6648
Best parameters:  {'C': 1.2}


#### XGBoostClassifier

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
param_grid = {'learning_rate': [0.1,0.2]}
grid = GridSearchCV(XGBClassifier(n_estimators=400, max_depth=5, min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8, objective='multi:softmax', num_class=3, scale_pos_weight = 5), param_grid, cv=5, scoring='f1_macro')

In [41]:
## CountVectorizer
xg_cv = grid.fit(X_train_cvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'CountVectorizer','ML_Model':'XGBoostClassifier','Mean_Score':grid.best_score_}, ignore_index=True) 

Best cross-validation f1_macro score: 0.7151
Best parameters:  {'learning_rate': 0.2}


In [42]:
## TfidfVectorizer
xg_tf = grid.fit(X_train_tvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'TfidfVectorizer','ML_Model':'XGBoostClassifier','Mean_Score':grid.best_score_}, ignore_index=True) 


Best cross-validation f1_macro score: 0.6897
Best parameters:  {'learning_rate': 0.2}


In [29]:
## Bag of Word
xg_bow = grid.fit(X_train_bow, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'Bag of Word','ML_Model':'XGBoostClassifier','Mean_Score':grid.best_score_}, ignore_index=True) 

Best cross-validation f1_macro score: 0.6991


In [ ]:
## HashingVectorizer
xg_hv = grid.fit(X_train_hvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'HashingVectorizer','ML_Model':'XGBoostClassifier','Mean_Score':grid.best_score_}, ignore_index=True) 

Best cross-validation f1_macro score: 0.6467


#### Naive Bayes

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
param_grid = {}
grid = GridSearchCV(GaussianNB(max), param_grid, cv=5, scoring='f1_macro')

In [13]:
## CountVectorizer

Cvect_nb = CountVectorizer(analyzer='word')
X_train_cvect_nb= Cvect.fit_transform(text_train)
X_test_cvect_nb = Cvect.transform(text_test)

nb_cv = cross_val_score(GaussianNB(), X_train_cvect_nb.toarray(), y_train, cv=5, scoring='f1_macro')
print("Mean Cross-validation f1_macro score: {:.4f}".format(np.mean(nb_cv)))
allmodelscorelist = allmodelscorelist.append({'PreProcessing':'CountVectorizer','ML_Model':'Naive Bayes','Mean_Score':np.mean(nb_cv)}, ignore_index=True) 

Mean Cross-validation f1_macro score: 0.5560


In [23]:
allmodelscorelist = allmodelscorelist.drop_duplicates()
print(allmodelscorelist)

       PreProcessing            ML_Model Mean_Score
0    CountVectorizer  LogisticRegression     0.7163
1    TfidfVectorizer  LogisticRegression     0.6810
2        Bag of Word  LogisticRegression     0.7163
3  HashingVectorizer  LogisticRegression     0.6648
4    CountVectorizer   XGBoostClassifier     0.7151
5    TfidfVectorizer   XGBoostClassifier     0.6897
6        Bag of Word   XGBoostClassifier     0.6991
7  HashingVectorizer   XGBoostClassifier     0.6467
8    CountVectorizer         Naive Bayes     0.5560


Suprisingly!! Kaggle submitted, best score is without pre processing the data!


In [20]:
##Kaggle Submitted Model
text_train_kaggle, y_train = train_data['text'], train_data['Target']
text_test_kaggle = test_data['text']

from sklearn.feature_extraction.text import CountVectorizer
Cvect = CountVectorizer(analyzer='word', ngram_range=(1,2), max_features=80000)
X_train_cvect= Cvect.fit_transform(text_train_kaggle)
X_test_cvect = Cvect.transform(text_test_kaggle)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'C': [0.8,0.9,1.0,1.1,1.2]}
grid = GridSearchCV(LogisticRegression(max_iter=200,penalty= 'l2'), param_grid, cv=5, scoring='f1_macro')
Kaggle_lg_cv = grid.fit(X_train_cvect, y_train)
print("Best cross-validation f1_macro score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)


Best cross-validation f1_macro score: 0.7305
Best parameters:  {'C': 1.1}


In [22]:
y_pred = grid.predict(X_test_cvect)
y_id=np.arange(7322, 14642, 1) 
output = pd.DataFrame(y_pred,y_id)
output.to_excel("/content/drive/My Drive/Colab Notebooks/NLP/HW1/train/output1.0.xlsx")